<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/crewai_industry_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet neo4j crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.2 MB/s eta

In [2]:
import datetime
import getpass
import os
from typing import Type

from crewai import LLM, Agent, Crew, Process, Task
from crewai.tools import BaseTool
from neo4j import GraphDatabase
from pydantic import BaseModel, Field


# Implementing an automated report generation agent
## Build dynamic, data-driven reports automatically using CrewAI and Neo4j

I've am a fan of agentic flows with LLMs. They not only enable more advanced text2cypher implementations but also open the door to various semantic layer implementations. It's an incredibly powerful and versatile approach.
In this blog post, I set out to implement a different kind of agent. Instead of the usual question-and-answer use case, this agent is designed to generate detailed reports about specific industries in a given location. The implementation leverages CrewAI, a platform that empowers developers to orchestrate AI agents with ease.

![image](https://cdn-images-1.medium.com/max/1600/1*27z9dMZqW2YXppDQDeH2Pg.png)

This system orchestrates three agents working in harmony to deliver a comprehensive business report:
* Data Researcher Agent: Specializes in gathering and analyzing industry-specific data for organizations in a given city, providing insights into company counts, public companies, combined revenue, and top-performing organizations.
* News Analyst Agent: Focuses on extracting and summarizing the latest news about relevant companies, offering a snapshot of trends, market movements, and sentiment analysis.
* Report Writer Agent: Synthesizes the research and news insights into a well-structured, actionable markdown report, ensuring clarity and precision without adding any unsupported information.

Together, these agents form a flow for generating insightful industry reports tailored to specific locations.

## Dataset
We will utilize the companies database available on the Neo4j demo server, which includes detailed information about organizations, individuals, and even the latest news for some of these organizations.

![image](https://cdn-images-1.medium.com/max/1600/1*IGxHIym9gfq-NtrdLGmGvw.png)

The dataset focuses on details such as investors, board members, and related aspects, making it an excellent resource for demonstrating industry report generation.


In [3]:
# Neo4j connection setup
URI = "neo4j+s://demo.neo4jlabs.com"
AUTH = ("companies", "companies")
DATABSE = "companies"
driver = GraphDatabase.driver(URI, auth=AUTH)

Next, we need to define the OpenAI key as we will be using gpt-4o throughout this blog post.

In [4]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI key: ")
llm = LLM(model='gpt-4o', temperature=0)

OpenAI key: ··········


## Knowledge-graph based tools
We will begin by implementing tools that enable an agent/LLM to retrieve relevant information from the database. The first tool will focus on fetching key statistics about companies within a specific industry in a given city.

In [5]:
industry_options = ["Software Companies", "Professional Service Companies", "Enterprise Software Companies", "Manufacturing Companies", "Software As A Service Companies", "Computer Hardware Companies", "Media And Information Companies", "Financial Services Companies", "Artificial Intelligence Companies", "Advertising Companies"]


class GetCityInfoInput(BaseModel):
    """Input schema for MyCustomTool."""
    city: str = Field(..., description="City name")
    industry: str = Field(..., description=f"Industry name, available options are: {industry_options}")

class GetCityInfo(BaseTool):
    name: str = "Get information about a specific city"
    description: str = "You can use this tools when you want to find information about specific industry within a city."
    args_schema: Type[BaseModel] = GetCityInfoInput

    def _run(self, city: str, industry: str) -> str:
        data, _, _ = driver.execute_query("""MATCH (c:City)<-[:IN_CITY]-(o:Organization)-[:HAS_CATEGORY]->(i:IndustryCategory)
WHERE c.name = $city AND i.name = $industry
WITH o
ORDER BY o.nbrEmployees DESC
RETURN count(o) AS organizationCount,
     sum(CASE WHEN o.isPublic THEN 1 ELSE 0 END) AS publicCompanies,
     sum(o.revenue) AS combinedRevenue,
     collect(CASE WHEN o.nbrEmployees IS NOT NULL THEN o END)[..5] AS topFiveOrganizations""", city=city, industry=industry)
        return [el.data() for el in data]

The GetCityInfo tool retrieves key statistics about companies in a specific industry within a given city. It provides information such as the total number of organizations, the count of public companies, combined revenue, and details about the top five organizations by employee size. This tool could be expanded, but for the purpose of this blog post I kept it simple.

The second tool can be used to fetch the latest information about a given company.

In [6]:
class GetNews(BaseTool):
    name: str = "Get the latest news for a specific company"
    description: str = "You can use this tool when you want to find the latest news about specific company"

    def _run(self, company: str) -> str:
        data, _, _ = driver.execute_query("""MATCH (c:Chunk)<-[:HAS_CHUNK]-(a:Article)-[:MENTIONS]->(o:Organization)
WHERE o.name = $company AND a.date IS NOT NULL
WITH c, a
ORDER BY a.date DESC
LIMIT 5
RETURN a.title AS title, a.date AS date, a.sentiment AS sentiment, collect(c.text) AS chunks""", company=company)
        return [el.data() for el in data]

The GetNews tool retrieves the latest news about a specific company. It provides details such as article titles, publication dates, sentiment analysis, and key excerpts from the articles. This tool is ideal for staying updated on recent developments and market trends related to a particular organization, allowing us to generate more detailed summaries.
## Agents
As mentioned, we will implement three agents. With CrewAI, minimal prompt engineering is required, as the platform handles the rest.
The agents are implemented as follows:

In [9]:
# Define Agents
class ReportAgents:
    def __init__(self):
        self.researcher = Agent(
            role='Data Researcher',
            goal='Gather comprehensive information about specific companies that are in relevant cities and industries',
            backstory="""You are an expert data researcher with deep knowledge of
            business ecosystems and city demographics. You excel at analyzing
            complex data relationships.""",
            verbose=True,
            allow_delegation=False,
            tools=[GetCityInfo()],
            llm=llm
        )

        self.news_analyst = Agent(
            role='News Analyst',
            goal='Find and analyze recent news about relevant companies in the specified industry and city',
            backstory="""You are a seasoned news analyst with expertise in
            business journalism and market research. You can identify key trends
            and developments from news articles.""",
            verbose=True,
            allow_delegation=False,
            tools=[GetNews()],
            llm=llm
        )

        self.report_writer = Agent(
            role='Report Writer',
            goal='Create comprehensive, well-structured reports combining the provided research and news analysis. Do not include any information that isnt explicitly provided.',
            backstory="""You are a professional report writer with experience in
            business intelligence and market analysis. You excel at synthesizing
            information into clear, actionable insights. Do not include any information that isn't explicitly provided.""",
            verbose=True,
            allow_delegation=False,
            llm=llm
        )

In CrewAI, agents are defined by specifying their role, goal, and backstory, with optional tools to enhance their capabilities. In this setup, three agents are implemented: a Data Researcher responsible for gathering detailed information about companies in specific cities and industries using the GetCityInfo tool; a News Analyst tasked with analyzing recent news about relevant companies leveraging the GetNews tool; and a Report Writer, who synthesizes the gathered information and news into a structured, actionable report without relying on external tools. This clear definition of roles and objectives ensures effective collaboration among the agents.
## Tasks
In addition to defining the agents, we also need to outline the tasks they will tackle. In this case, we will define three distinct tasks.

In [10]:
def generate_report(city_name: str, industry_name: str):
    # Initialize components
    agents = ReportAgents()

    # Define Tasks
    city_research_task = Task(
        description=f"""Research and analyze {city_name} and its business ecosystem in {industry_name} industry:
        1. Get city summary and key information
        2. Find organizations in the specified industry
        3. Analyze business relationships and economic indicators""",
        agent=agents.researcher,
        expected_output="Basic statistics about the companies in the given city and industry as well as top performers"
    )

    news_analysis_task = Task(
        description=f"""Analyze recent news about the companies provided by the city researcher""",
        agent=agents.news_analyst,
        expected_output="Summarization of the latest news for the company and how it might affect the market",
        context=[city_research_task]

    )

    report_writing_task = Task(
        description=f"""Create a detailed markdown report about the
        results you got from city research and news analysis tasks.
        Do not include any information that isn't provided""",
        agent=agents.report_writer,
        expected_output="Markdown summary",
        context=[city_research_task, news_analysis_task]

    )

    # Create and run the crew
    crew = Crew(
        agents=[agents.researcher, agents.news_analyst, agents.report_writer],
        tasks=[city_research_task, news_analysis_task, report_writing_task],
        verbose=True,
        process=Process.sequential,

    )

    result = crew.kickoff()
    return result

The tasks are designed to align with the agents' capabilities. The city research task focuses on analyzing the business ecosystem of a specified city and industry, gathering key statistics and identifying top-performing organizations, handled by the Data Researcher. The news analysis task examines recent developments related to these companies, summarizing key trends and market impacts, using the output from city research and performed by the News Analyst. Finally, the report writing task synthesizes the findings from the previous tasks into a comprehensive markdown report, completed by the Report Writer.

Let's test it!

In [11]:
city = "Seattle"
industry = "Hardware Companies"
report = generate_report(city, industry)
print(report)


# Agent: Data Researcher
## Task: Research and analyze Seattle and its business ecosystem in Hardware Companies industry:
        1. Get city summary and key information
        2. Find organizations in the specified industry
        3. Analyze business relationships and economic indicators




# Agent: Data Researcher
## Using tool: Get information about a specific city
## Tool Input: 
"{\"city\": \"Seattle\", \"industry\": \"Computer Hardware Companies\"}"
## Tool Output: 
[{'organizationCount': 24, 'publicCompanies': 4, 'combinedRevenue': 229136415000.0, 'topFiveOrganizations': [{'summary': 'American multinational technology corporation', 'revenue': 198270000000.0, 'isDissolved': False, 'diffbotId': 'https://diffbot.com/entity/EIsFKrN_ZNLSWsvxdQfWutQ', 'nbrEmployees': 221000, 'name': 'Microsoft Corporation', 'motto': 'We’re on a mission to empower every person and every organization on the planet to achieve more.', 'isPublic': True, 'id': 'EIsFKrN_ZNLSWsvxdQfWutQ'}, {'summary': 'American multinational technology company', 'revenue': 26974000000.0, 'isDissolved': False, 'diffbotId': 'https://diffbot.com/entity/E9nQaOvC9MXaUzXCuN81tFQ', 'nbrEmployees': 26196, 'name': 'Nvidia Corporation', 'isPublic': True, 'motto': 'Formerly Mellanox Technologies and Cumulus Networks', 'id

_Note that the demo dataset is outdated as we don't import news regularly._
## Summary
Building an automated report generation pipeline using agentic flows, Neo4j, and CrewAI offers a glimpse into how LLMs can move beyond simple question-and-answer interactions. By assigning specialized tasks to a suite of agents, and arming them with the right tools, we can orchestrate a dynamic, exploratory workflow that pulls relevant data, processes it, and composes well-structured insights.

Through this approach, agents collaborate to uncover key statistics about an industry in a given city, gather the latest news, and synthesize everything into a polished markdown report. This goes to show that LLMs can be deployed in creative, multi-step processes, enabling more sophisticated use cases like automated business intelligence, data-driven content creation, and beyond.